## Import Library

In [1]:
import sys
sys.path.insert(
    1,
    '/home/dtd/Documents/PhD_Life/Source Code/my_work/lib'
)

sys.path.insert(
    1,
    '/home/dtd/Documents/PhD_Life/Source Code/my_work/config'
)

import numpy as np
import pandas as pd
import logging
import incremental_ps_score_estimator as ipse
import math
import timeit
import utils
import experiment_config as cf

from tqdm import tqdm

import dowhy.datasets
import dowhy
from dowhy import CausalModel

from math import sqrt
from econml.drlearner import ForestDRLearner, LinearDRLearner
from econml.metalearners import SLearner, XLearner, TLearner
from econml.ortho_forest import CausalTree, ContinuousTreatmentOrthoForest, DiscreteTreatmentOrthoForest
from econml.dml import ForestDMLCateEstimator, LinearDMLCateEstimator, SparseLinearDMLCateEstimator
from econml.inference import BootstrapInference
from econml.sklearn_extensions.linear_model import WeightedLasso, WeightedLassoCV

### Import sklearn
from scipy.stats import sem
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import explained_variance_score, mean_squared_error
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, FunctionTransformer

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

from zepid.causal.doublyrobust import TMLE
# from cforest.forest import CausalForest
from bartpy.sklearnmodel import SklearnModel


/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


ModuleNotFoundError: No module named 'cforest'

## Parameter 

In [ ]:
treatment = 't'
outcome = 'yf'
col =  ["t", "yf", "ycf", "mu0", "mu1" ]
cov = ["x" + str(i) for i in range(1,26)]
col = col + cov
features = cov + ["t"]

RESULT_PATH = "/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/result"

In [6]:
# PATH = "/home/dtd/Downloads/dragonnet/data_train_test"
# for index_ in tqdm(range(1,101)):    
#     data = utils.load_data(PATH_TRAIN, index_)
#     data_test = utils.load_data(PATH_TEST, index_)
#     data.to_csv(PATH + "/train_ihdp_{}",format(index_),encoding='utf-8')
#     data_test.to_csv(PATH + "/test_ihdp_{}",format(index_), encoding='utf-8')

## Compare with EconML

In [8]:
df_train_result = pd.DataFrame(index=range(100))
df_train_result['mae_linear_ortho'] = 0
df_train_result['mae_linear_dr'] = 0
df_train_result['mae_forest_dr'] = 0
df_train_result['mae_x_leaner'] = 0
df_train_result['mae_s_leaner'] = 0
df_train_result['mae_t_leaner'] = 0
df_train_result['incremental'] = 0
df_train_result['incremental_pehe'] = 0
df_train_result['causal_forest'] = 0


df_test_result = pd.DataFrame(index=range(100))
df_test_result['mae_linear_ortho'] = 0
df_test_result['mae_linear_dr'] = 0
df_test_result['mae_forest_dr'] = 0
df_test_result['mae_x_leaner'] = 0
df_test_result['mae_s_leaner'] = 0
df_test_result['mae_t_leaner'] = 0
df_test_result['incremental'] = 0
df_test_result['incremental_pehe'] = 0
df_test_result['causal_forest'] = 0



### Causal Forest

In [6]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"

for index_ in tqdm(range(1,101)):    
    data = utils.load_data(PATH_TRAIN, index_)
    data_test = utils.load_data(PATH_TEST, index_)
    
    data['term_t'] = data[treatment].apply(lambda x: bool(x))
    data_test['term_t'] = data_test[treatment].apply(lambda x: bool(x))
    
    cf = CausalForest(
        num_trees=50,
        split_ratio=0.5,
        min_leaf=5,
        max_depth=20,
        use_transformed_outcomes=True,
        num_workers=4,
        seed_counter=1,
    )

    cf = cf.fit(data[features].values, data['term_t'].values, data[outcome].values)

    ate_t_learner_train = np.mean(cf.predict(data[features].values))
    ate_t_learner_test = np.mean(cf.predict(data_test[features].values))
    
    
    true_effect = data['mu1'] - data['mu0']
    means_train, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
    
    true_effect = data_test['mu1'] - data_test['mu0']
    means_test, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)

    mae_t_leaner_train = utils.abs_ate(means_train, ate_t_learner_train)
    mae_t_leaner_test = utils.abs_ate(means_test, ate_t_learner_test)
    
    df_train_result.loc[index_ - 1, 'causal_forest'] = mae_t_leaner_train
    df_test_result.loc[index_ - 1, 'causal_forest'] = mae_t_leaner_test

100%|██████████| 100/100 [06:43<00:00,  4.03s/it]


In [5]:
print("MAE training {:.3f}".format(df_train_result.causal_forest.mean()))
print("MAE testing {:.3f}".format(df_test_result.causal_forest.mean()))

NameError: name 'df_train_result' is not defined

###  BART

In [8]:
def causal_bart(data, bart_model):
    data_pos = data.copy()
    data_pos[treatment] = 1
    data['treated_cf_outcome'] = bart_model.predict(data_pos[features])

    ## Compute counterfactual outcome with treatment
    data_neg = data.copy()
    data_neg[treatment] = 0
    data['control_cf_outcome'] = bart_model.predict(data_neg[features])
    
    return np.mean(data['treated_cf_outcome'] - data['control_cf_outcome'])

In [ ]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"

df_train_result['bart'] = 0
df_test_result['bart'] = 0

for index_ in tqdm(range(1,101)):    
    data = utils.load_data(PATH_TRAIN, index_)
    data_test = utils.load_data(PATH_TEST, index_)
    
    bart_model = SklearnModel() # Use default parameters
    bart_model.fit(data[features].values, data[outcome].values)
    
    effects_train = causal_bart(data, bart_model)
    effects_test = causal_bart(data_test, bart_model)

    true_effect = data['mu1'] - data['mu0']
    means_train, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
    
    true_effect = data_test['mu1'] - data_test['mu0']
    means_test, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
    
    mae_t_leaner_train = utils.abs_ate(means_train, effects_train)
    mae_t_leaner_test = utils.abs_ate(means_test, effects_test)
    
    
    df_train_result.loc[index_ - 1, 'bart'] = mae_t_leaner_train
    df_test_result.loc[index_ - 1, 'bart'] = mae_t_leaner_test
    

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
print("BART training {:.3f}".format(df_train_result.causal_forest.mean()))
print("BART testing {:.3f}".format(df_test_result.causal_forest.mean()))

### Run EconML

#### T Leaner

In [ ]:
# PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
# PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"

# for index_ in tqdm(range(1,101)):    
#     data = utils.load_data(PATH_TRAIN, index_)
#     data_test = utils.load_data(PATH_TEST, index_)
    
#     true_effect = data['mu1'] - data['mu0']
#     means_train, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
    
#     true_effect = data_test['mu1'] - data_test['mu0']
#     means_test, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
#     ## T-Learner
#     est = TLearner(models=GradientBoostingRegressor())
#     est.fit(data[outcome].values, data[treatment].values, data[features].values)
    
#     ate_t_learner_train = np.mean(est.effect(data[features].values))
#     ate_t_learner_test = np.mean(est.effect(data_test[features].values))
#     mae_t_leaner_train = utils.abs_ate(means_train, ate_t_learner_train)
#     mae_t_leaner_test = utils.abs_ate(means_test, ate_t_learner_test)
#     df_train_result.loc[index_ - 1, 'mae_t_leaner'] = round(mae_t_leaner_train,2)
#     df_test_result.loc[index_ - 1, 'mae_t_leaner'] = round(mae_t_leaner_test,2)

In [ ]:
df_train_result.mae_t_leaner.mean(), df_test_result.mae_t_leaner.mean()

#### Others

In [ ]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"

for index_ in tqdm(range(1,101)):    
    data = utils.load_data(PATH_TRAIN, index_)
    data_test = utils.load_data(PATH_TEST, index_)
    
    """
    ## Double ML
    est = LinearDMLCateEstimator(model_y=LassoCV(), model_t=LassoCV())
    ### Estimate with OLS confidence intervals
    est.fit(Y = data[outcome].values, T = data[treatment].values, X = data[features].values, inference='statsmodels')
    ate_dr_train = np.mean(est.effect(data[features].values))
    print(ate_dr_train)

    est = SparseLinearDMLCateEstimator(model_y=LassoCV(), model_t=LassoCV())
    ### Estimate with OLS confidence intervals
    est.fit(Y = data[outcome].values, T = data[treatment].values, X = data[features].values, inference='statsmodels')
    ate_dr_train = np.mean(est.effect(data[features].values))
    print(ate_dr_train)
    """
    
    ## Linear
    est = LinearDRLearner(model_propensity=GradientBoostingClassifier(),
                          model_regression=GradientBoostingRegressor())
    est.fit(Y = data[outcome].values, T = data[treatment].values, X = data[features].values, inference='statsmodels')
    ate_dr_train = np.mean(est.effect(data[features].values))
    ate_dr_test = np.mean(est.effect(data_test[features].values))
    
    ## Forest
    est = ForestDRLearner(model_propensity=GradientBoostingClassifier(),
                      model_regression=GradientBoostingRegressor())
    est.fit(Y = data[outcome].values, T = data[treatment].values, X = data[features].values, inference='blb')
    ate_fdr_train = np.mean(est.effect(data[features].values))
    ate_fdr_test = np.mean(est.effect(data_test[features].values))
    
    """
    ## X Learner
    est = XLearner(models=GradientBoostingRegressor(),
              propensity_model=GradientBoostingClassifier(),
              cate_models=GradientBoostingRegressor())
    est.fit(data[outcome].values, data[treatment].values, data[features].values)

    ate_x_learner_train = np.mean(est.effect(data[features].values))
    ate_x_learner_test = np.mean(est.effect(data_test[features].values))
    
    ## S-Learner
    est = SLearner(overall_model=GradientBoostingRegressor())
    est.fit(data[outcome].values, data[treatment].values, data[features].values)
    
    ate_s_learner_train = np.mean(est.effect(data[features].values))
    ate_s_learner_test = np.mean(est.effect(data_test[features].values))
    
    
    ## T-Learner
    
    
    est = TLearner(models=GradientBoostingRegressor())
    est = TLearner(models=LassoCV())
    est.fit(data[outcome].values, data[treatment].values, data[features].values)
    
    ate_t_learner_train = np.mean(est.effect(data[features].values))
    ate_t_learner_test = np.mean(est.effect(data_test[features].values))
    
    """
    ## OrthoForest
    est = DiscreteTreatmentOrthoForest(n_trees=500,
                                   model_Y = WeightedLasso(alpha=0.5))

    est.fit(Y = data[outcome].values, T = data[treatment].values, X = data[features].values, inference='blb')
    
    ate_ortho_train = np.mean(est.effect(data[features].values))
    ate_ortho_test = np.mean(est.effect(data_test[features].values))
    
    ### df_train
    true_effect = data['mu1'] - data['mu0']
    means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
    
    mae_linear_ortho = utils.abs_ate(means, ate_ortho_train)
    mae_linear_dr = utils.abs_ate(means, ate_dr_train)
    mae_forest_dr = utils.abs_ate(means, ate_fdr_train)
    #mae_x_leaner = utils.abs_ate(means, ate_x_learner_train)
    #mae_s_leaner = utils.abs_ate(means, ate_s_learner_train)
    #mae_t_leaner = utils.abs_ate(means, ate_t_learner_train)
    

    df_train_result.loc[index_ - 1, 'mae_linear_ortho'] = round(mae_linear_ortho,2)
    df_train_result.loc[index_ - 1, 'mae_linear_dr'] = round(mae_linear_dr,2)
    df_train_result.loc[index_ - 1, 'mae_forest_dr'] = round(mae_forest_dr,2)
    #df_train_result.loc[index_ - 1, 'mae_x_leaner'] = round(mae_x_leaner,2)
    #df_train_result.loc[index_ - 1, 'mae_s_leaner'] = round(mae_s_leaner,2)
    #df_train_result.loc[index_ - 1, 'mae_t_leaner'] = round(mae_t_leaner,2)

    

    ## df_test
    true_effect = data_test['mu1'] - data_test['mu0']
    means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
    
    mae_linear_ortho = utils.abs_ate(means, ate_ortho_test)
    mae_linear_dr = utils.abs_ate(means, ate_dr_test)
    mae_forest_dr = utils.abs_ate(means, ate_fdr_test)
    #mae_x_leaner = utils.abs_ate(means, ate_x_learner_test)
    #mae_s_leaner = utils.abs_ate(means, ate_s_learner_test)
    #mae_t_leaner = utils.abs_ate(means, ate_t_learner_test)
    
    df_test_result.loc[index_ - 1, 'mae_linear_ortho'] = round(mae_linear_ortho,2)
    df_test_result.loc[index_ - 1, 'mae_linear_dr'] = round(mae_linear_dr,2)
    df_test_result.loc[index_ - 1, 'mae_forest_dr'] = round(mae_forest_dr,2)
    #df_test_result.loc[index_ - 1, 'mae_x_leaner'] = round(mae_x_leaner,2)
    #df_test_result.loc[index_ - 1, 'mae_s_leaner'] = round(mae_s_leaner,2)
    #df_test_result.loc[index_ - 1, 'mae_t_leaner'] = round(mae_t_leaner,2)


### K-nearest neighbor + Linear Regression

In [ ]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"

df_train_result['mea_knn'] = 0
df_test_result['mea_knn'] = 0

df_train_result['linear_regression'] = 0
df_test_result['linear_regression'] = 0

for index_ in tqdm(range(1,101)):    
    data = utils.load_data(PATH_TRAIN, index_)
    data_test = utils.load_data(PATH_TEST, index_)
    
    model = CausalModel(data=data,
                        treatment=treatment,
                        outcome=outcome,
                        common_causes=cov, proceed_when_unidentifiable=True)
    data[treatment] = [bool(x) for x in data[treatment]]
    identified_estimand = model.identify_effect()

    estimate_we = model.estimate_effect(
            identified_estimand, method_name="backdoor.propensity_score_matching")

    estimate_linear = model.estimate_effect(
            identified_estimand, method_name="backdoor.linear_regression")
    
    
    model = CausalModel(data=data_test,
                        treatment=treatment,
                        outcome=outcome,
                        common_causes=cov, proceed_when_unidentifiable=True)
    data_test[treatment] = [bool(x) for x in data_test[treatment]]
    identified_estimand = model.identify_effect()

    estimate_we_test = model.estimate_effect(
        identified_estimand, method_name="backdoor.propensity_score_matching")
    
    estimate_linear_test = model.estimate_effect(
            identified_estimand, method_name="backdoor.linear_regression")
    
    true_effect = data['mu1'] - data['mu0']
    means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
    true_effect = data_test['mu1'] - data_test['mu0']
    means_test, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
        
    mae_incre_train = utils.abs_ate(means, estimate_we.value)
    mae_incre_test = utils.abs_ate(means_test, estimate_we_test.value)
    df_train_result.loc[index_ - 1, 'mea_knn'] = mae_incre_train
    df_test_result.loc[index_ - 1, 'mea_knn'] = mae_incre_test
    
    mae_incre_train = utils.abs_ate(means, estimate_linear.value)
    mae_incre_test = utils.abs_ate(means_test, estimate_linear_test.value)
    df_train_result.loc[index_ - 1, 'linear_regression'] = mae_incre_train
    df_test_result.loc[index_ - 1, 'linear_regression'] = mae_incre_test
    

In [ ]:
print(df_train_result.mea_knn.mean())
print(df_test_result.mea_knn.mean())

### TMLE

In [ ]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"

expose_m = ' + '.join(cov)
outcome_m = ' + '.join(features)

df_train_result['mea_tmle'] = 0
df_test_result['mea_tmle'] = 0

for index_ in tqdm(range(1,101)):    
    data = utils.load_data(PATH_TRAIN, index_)
    data_test = utils.load_data(PATH_TEST, index_)
    model_t = LogisticRegression(penalty='l2', random_state=201)
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    
    tml = TMLE(data, exposure=treatment, outcome=outcome)
    tml.exposure_model(expose_m, bound=[0.01, 0.99], print_results=False, custom_model=model_t)
    tml.outcome_model(outcome_m, print_results=False, custom_model = model_y)
    tml.fit()
    
    true_effect = data['mu1'] - data['mu0']
    means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
    mae_train = utils.abs_ate(means, tml.average_treatment_effect)

    tml = TMLE(data_test, exposure=treatment, outcome=outcome)
    tml.exposure_model(expose_m, bound=[0.01, 0.99], print_results=False, custom_model=model_t)
    tml.outcome_model(outcome_m, print_results=False)
    tml.fit()
    
    true_effect = data_test['mu1'] - data_test['mu0']
    means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
    mae_test = utils.abs_ate(means, tml.average_treatment_effect)
        
    df_train_result.loc[index_ - 1, 'mea_tmle'] = mae_train
    df_test_result.loc[index_ - 1, 'mea_tmle'] = mae_train

### Incremental propensity score

In [9]:
# cost function for base models
def rmse(yreal, yhat):
    return sqrt(mean_squared_error(yreal, yhat))
def get_super_learner_outcome(X):
    ensemble = SuperLearner(scorer=rmse, folds=10, shuffle=True, sample_size=len(X))
    
    models = list()
    models.append(LinearRegression())
    models.append(ElasticNet())
    models.append(SVR(gamma='scale'))
    models.append(DecisionTreeRegressor())
    models.append(GradientBoostingRegressor(random_state=0, n_estimators = 5000))
    models.append(AdaBoostRegressor())
    models.append(BaggingRegressor(n_estimators=100))
    models.append(RandomForestRegressor(n_estimators=100))
    models.append(ExtraTreesRegressor(n_estimators=100))
    ensemble.add(models)
    ensemble.add_meta(LinearRegression())
    return ensemble

def f1(y, p): return f1_score(y, p, average='micro')


def get_super_learner_treatment(X):
    ensemble = SuperLearner(scorer=f1, random_state=1)
    
    models = list()
    models.append(KNeighborsClassifier(3))
    models.append(RandomForestClassifier(n_estimators=30, random_state=1))
    models.append(GradientBoostingClassifier())
    models.append(SVC())
    models.append(AdaBoostClassifier())
    models.append(MLPClassifier(alpha=1, max_iter=1000))
    models.append(GaussianProcessClassifier(1.0 * RBF(1.0)))
    

    ensemble.add(models)
    ensemble.add_meta(LogisticRegression())
    return ensemble



### Single delta

In [12]:
delta_seq = np.load('delta.npy')

PATH_TRAIN = "/home/dtd/Documents/data/idhp/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/data/idhp/ihdp_npci_1-100.test.npz"

mae_incre_seq  = []
rmse  = []
rmse1  = []
rmse2  = []

for index_ in tqdm(range(1,101)):    
    data = utils.load_data(PATH_TRAIN, index_)
    data_test = utils.load_data(PATH_TEST, index_)

    #Incremental propensity score
    ## Fit treatment
    model_t = LogisticRegression()
    model_t.fit(data[cov], data[treatment])
    

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y.fit(data[features].values, data[outcome].values)
    y_pred = model_y.predict(data_test[features].values)
    rmse.append(mean_squared_error(data_test[outcome].values, 
                                       y_pred,
                                       sample_weight=None, 
                                       multioutput='uniform_average', 
                                       squared=False))

    delta = delta_seq[index_ - 1]
    
    ## df_train
    influence = ipse.influence_function(data, treatment, cov, outcome, features, delta, model_y, model_t)
    means_incre, stds_incre = np.mean(influence, axis=0), sem(influence, axis=0)
    
    
    true_effect = data['mu1'] - data['mu0']
    means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
    
    mae_incre = utils.abs_ate(means, means_incre)
    
    df_train_result.loc[index_ - 1, 'incremental'] = mae_incre
    pehe = utils.pehe(data['mu1'], data['mu0'], data['cf1'], data['cf0'])
    print("Index ", index_)
    print("MAE {:.4f}, PEHE {:.4f}".format(mae_incre, pehe))
    
    df_train_result.loc[index_ - 1, 'incremental_pehe'] = pehe
    
    ## df_test
    influence = ipse.influence_function(data_test, treatment, cov, outcome, features, delta, model_y, model_t)
    means_incre, stds_incre = np.mean(influence, axis=0), sem(influence, axis=0)

    true_effect = data_test['mu1'] - data_test['mu0']
    means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
    
    mae_incre = utils.abs_ate(means, means_incre)
    pehe = utils.pehe(data_test['mu1'], data_test['mu0'], data_test['cf1'], data_test['cf0'])
    
    df_test_result.loc[index_ - 1, 'incremental'] = mae_incre
    df_test_result.loc[index_ - 1, 'incremental_pehe'] = pehe
    
    print("MAE {:.4f}, PEHE {:.4f}".format(mae_incre, pehe))

    print("*"*100)

  1%|          | 1/100 [00:04<07:45,  4.70s/it]

Index  1
MAE 0.1555, PEHE 0.5797
MAE 0.1871, PEHE 0.5364
****************************************************************************************************


  2%|▏         | 2/100 [00:09<07:40,  4.70s/it]

Index  2
MAE 0.2209, PEHE 0.4765
MAE 0.1265, PEHE 0.4283
****************************************************************************************************


  3%|▎         | 3/100 [00:14<07:39,  4.74s/it]

...

Index  98
MAE 0.0010, PEHE 4.5816
MAE 0.6522, PEHE 7.0810
****************************************************************************************************


 99%|█████████▉| 99/100 [08:04<00:04,  4.77s/it]

Index  99
MAE 0.1012, PEHE 1.7562
MAE 0.0572, PEHE 1.3023
****************************************************************************************************


100%|██████████| 100/100 [08:08<00:00,  4.89s/it]

Index  100
MAE 0.3384, PEHE 0.6815
MAE 0.0747, PEHE 0.8508
****************************************************************************************************


In [15]:
print("MAE - Incremental ", np.mean(df_train_result.incremental.mean()))
print("MAE - Incremental ", np.mean(df_test_result.incremental.mean()))
print("PEHE - Incremental ", np.mean(df_train_result.incremental_pehe.mean()))
print("PHEE - Incremental ", np.mean(df_test_result.incremental_pehe.mean()))


MAE - Incremental  0.1842470026240425
MAE - Incremental  0.6930728889683614
PEHE - Incremental  2.5191639927754483
PHEE - Incremental  2.7597990630576454


In [ ]:
# MAE - Incremental  0.42532114248173786
# MAE - Incremental  0.5544695165630058
## Gradient boosting
# MAE - Incremental  0.2173673527217913
# MAE - Incremental  0.6196024525840684


In [ ]:
# cfrnet = (0.299  + 0.262)/2

### Continuous delta

In [28]:
delta_seq[1]

IndexError: index 1 is out of bounds for axis 0 with size 1

In [33]:
delta_seq = np.load('list_delta_.npy')
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"


for index_ in tqdm(range(1,10)):    
    data = utils.load_data(PATH_TRAIN, index_)
    data_test = utils.load_data(PATH_TEST, index_)

    #Incremental propensity score
    ## Fit treatment
    model_t = LogisticRegression()
    model_t.fit(data[cov], data[treatment])
    

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y.fit(data[features], data[outcome])

    data['p1'] = model_t.predict_proba(data[cov])[:,1]
    data_test['p1'] = model_t.predict_proba(data_test[cov])[:,1]

    treated_neighbors = (
            NearestNeighbors(n_neighbors=1, algorithm='ball_tree')
            .fit(data['p1'].values.reshape(-1, 1))
    )
    distances, indices = treated_neighbors.kneighbors(data_test['p1'].values.reshape(-1, 1))
    
    """
    print(data.loc[indices[0],'p1'], data_test.loc[0,'p1'])
    print(data.loc[indices[1],'p1'], data_test.loc[1,'p1'])
    print(data.loc[indices[2],'p1'], data_test.loc[2,'p1'])
    print(data.loc[indices[3],'p1'], data_test.loc[3,'p1'])
    print(data.loc[indices[73],'p1'], data_test.loc[73,'p1'])
    print(data.loc[indices[74],'p1'], data_test.loc[74,'p1'])
    """
    
    delta = delta_seq[index_ - 1]
    
    ## df_train
    influence = ipse.influence_function(data, treatment, cov, outcome, features, delta, model_y, model_t)
    means_incre, stds_incre = np.mean(influence, axis=0), sem(influence, axis=0)

    true_effect = data['mu1'] - data['mu0']
    means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
    
    mae_incre = utils.abs_ate(means, means_incre)
    #print("Data train index {}. MAE {:.2f}".format(index_, mae_incre))

    df_train_result.loc[index_ - 1, 'incremental_con'] = mae_incre
    
    ## df_test
    delta = delta_seq[index_ - 1][indices.reshape(-1)]
    influence = ipse.influence_function(data_test, treatment, cov, outcome, features, delta, model_y, model_t)
    means_incre, stds_incre = np.mean(influence, axis=0), sem(influence, axis=0)

    true_effect = data_test['mu1'] - data_test['mu0']
    means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
    
    mae_incre = utils.abs_ate(means, means_incre)
    
    df_test_result.loc[index_ - 1, 'incremental_con'] = mae_incre
    
    #print("Data test index {}. MAE {:.2f}".format(index_, mae_incre))


100%|██████████| 9/9 [00:42<00:00,  4.69s/it]


In [34]:
df_train_result.incremental_con.mean(), df_test_result.incremental_con.mean()

### Counterfactual Representation

In [11]:
mmd_train = 0.282
mmd_test = 0.299
wass_train  = 0.255
wass_test  = 0.262
bnn_0_4_train = (1.445 + 2.021) / 2
bnn_0_4_test = (1.240 + 1.791) / 2
bnn_2_2_train = 0.229
bnn_2_2_test = 0.261


## Conclusion

In [12]:
RESULT_PATH = "/home/dtd/Documents/PhD_Life/Source Code/my_work/result/idhp"
df_train_result = pd.read_csv(RESULT_PATH + "/train_result.csv")
df_test_result = pd.read_csv(RESULT_PATH + "/test_result.csv")

training_result = {}
training_result["Linear_Regression"] = [df_train_result.linear_regression.mean(),
                                        df_train_result.linear_regression.std()
                                       ]
training_result["K-NN"] = [df_train_result.mea_knn.mean(),
                           df_train_result.mea_knn.std()
                          ]
training_result["Incremental_Con_PS"] = [df_train_result.incremental_con.mean(),
                                         df_train_result.incremental_con.std()
                                        ]
training_result["CFR_WASS"] = [wass_train, 0.032]
training_result["CFR_MMD"] = [mmd_train, 0.034]
training_result["Incremental_PS"]= [df_train_result.incremental.mean(),
                                    df_train_result.incremental.std()
                                   ]
training_result["OrthoForest"] = [df_train_result.mae_linear_ortho.mean(),
                                  df_train_result.mae_linear_ortho.std()
                                 ]
training_result["ForestDRLearner"] = [df_train_result.mae_forest_dr.mean(),
                                      df_train_result.mae_forest_dr.std()
                                     ]
training_result["LinearDRLearner"] = [df_train_result.mae_linear_ortho.mean(),
                                      df_train_result.mae_linear_ortho.std()
                                     ]
training_result["Bart"] = [df_train_result.bart.mean(),
                           df_train_result.bart.std()
                          ]
training_result["Causal_forest"] = [df_train_result.causal_forest.mean(),
                                    df_train_result.causal_forest.std()
                                   ]
training_result["BNN-2-2"] = [bnn_2_2_train, 0.03]
training_result["TMLE"] = [df_train_result.mea_tmle.mean(),
                                    df_train_result.mea_tmle.std()
                                   ]


test_result = {}
test_result["Linear_Regression"] = [df_test_result.linear_regression.mean(),
                                    df_test_result.linear_regression.std()
                                   ]
test_result["K-NN"] = [df_test_result.mea_knn.mean(),
                       df_test_result.mea_knn.std()
                      ]
test_result["Incremental_Con_PS"] = [df_test_result.incremental_con.mean(),
                                     df_test_result.incremental_con.std()
                                    ]
test_result["CFR_WASS"] = [wass_test, 0.034]
test_result["CFR_MMD"] = [mmd_test, 0.032]
test_result["Incremental_PS"]= [df_test_result.incremental.mean(),
                                df_test_result.incremental.std()
                               ]
test_result["OrthoForest"] = [df_test_result.mae_linear_ortho.mean(),
                              df_test_result.mae_linear_ortho.std()
                             ]
test_result["ForestDRLearner"] = [df_test_result.mae_forest_dr.mean(),
                                  df_test_result.mae_forest_dr.std()
                                 ]
test_result["LinearDRLearner"] = [df_test_result.mae_linear_ortho.mean(),
                                  df_test_result.mae_linear_ortho.std()
                                 ]
test_result["Bart"] = [df_test_result.bart.mean(),
                       df_test_result.bart.std()
                      ]
test_result["Causal_forest"] = [df_test_result.causal_forest.mean(),
                                df_test_result.causal_forest.std()
                               ]  
test_result["BNN-2-2"] =[bnn_2_2_test, 0.08]


In [13]:
training_result = {k: v for k, v in sorted(training_result.items(), key=lambda item: item[1][0])}
test_result = {k: v for k, v in sorted(test_result.items(), key=lambda item: item[1][0])}

In [14]:
print("Training set")
for k,v in training_result.items():
    print("\t", k, "{:.3f} +/- {:.3f}".format(v[0], v[1]))
    
print("Testing set")
for k,v in test_result.items():
    print("\t", k, "{:.3f} +/- {:.3f}".format(v[0], v[1]))

Training set
	 Incremental_Con_PS 0.046 +/- 0.012
	 Incremental_PS 0.184 +/- 0.440
	 K-NN 0.205 +/- 0.214
	 BNN-2-2 0.229 +/- 0.030
	 CFR_WASS 0.255 +/- 0.032
	 CFR_MMD 0.282 +/- 0.034
	 TMLE 0.329 +/- 0.616
	 Bart 0.709 +/- 1.225
	 Linear_Regression 0.746 +/- 1.398
	 Causal_forest 0.915 +/- 1.346
	 ForestDRLearner 2.294 +/- 1.029
	 OrthoForest 3.095 +/- 2.077
	 LinearDRLearner 3.095 +/- 2.077
Testing set
	 BNN-2-2 0.261 +/- 0.080
	 CFR_WASS 0.262 +/- 0.034
	 CFR_MMD 0.299 +/- 0.032
	 Incremental_PS 0.693 +/- 1.400
	 K-NN 0.734 +/- 0.836
	 Bart 0.767 +/- 1.098
	 Incremental_Con_PS 0.885 +/- 2.005
	 Causal_forest 1.181 +/- 2.332
	 Linear_Regression 1.264 +/- 2.646
	 ForestDRLearner 2.165 +/- 0.754
	 OrthoForest 2.983 +/- 1.877
	 LinearDRLearner 2.983 +/- 1.877


In [ ]:
PATH_RESULT = "/home/dtd/Documents/interpretable_machine_learning/Source Code/result/idhp"
df_train_result.to_csv(PATH_RESULT + "/train_result.csv", index = False)
df_test_result.to_csv(PATH_RESULT + "/test_result.csv", index = False)

## Predictive model

In [ ]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"

df_train = utils.load_data(PATH_TRAIN, 1)
df_test = utils.load_data(PATH_TEST, 1)

In [ ]:
df_train.head()

In [ ]:
_one_hot_encoder = OneHotEncoder(categories='auto', sparse=False)
_one_hot_encoder.fit(df_train.t.values.reshape((-1,1)))
t = _one_hot_encoder.transform(df_train.t.values.reshape((-1,1)))

In [ ]:
model_t = LogisticRegression()
model_t.fit(df_train[cov], t)